## Armut Association Rule Based Recommender System

Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır. Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca ulaşılmasını sağlamaktadır.

Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak Association Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.

Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır. Alınan her hizmetin tarih ve saat bilgisini içermektedir.

In [21]:
# UserId: Müşteri numarası
# ServiceId: Her kategoriye ait anonimleştirilmiş servislerdir. (Örnek : Temizlik kategorisi altında koltuk yıkama servisi)
# Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.
# (Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)
# CategoryId: Anonimleştirilmiş kategorilerdir. (Örnek : Temizlik, nakliyat, tadilat kategorisi)
# CreateDate: Hizmetin satın alındığı tarih


### GÖREV 1: Veriyi Hazırlama

In [22]:
#Kütüphanelerimizi intall edelim;
import pandas as pd
!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [23]:
# Verisetimizi getirip, daha sonrasında rahat bir şekilde çağırabilmek adına kopyasını oluşturalım;
df_= pd.read_csv("datasets/armut_data.csv")
df= df_.copy()
df.head()


,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


Adım 1: ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir. ServiceID ve CategoryID’yi "_" ile birleştirerek bu hizmetleri temsil edecek yeni bir değişken oluşturuyoruz.

In [24]:
# Service adında yeni bir değişken oluşturuyoruz. Bir list comprehension içerisinde for döngüsü yazzıyoruz. 
# Değerlerde gez, serviceıd1. ve 2. de gez - çizgi ile birleştir diyoruz.
df["Sevice"]= [str(row[1]) + str(row[2]) for row in df.values]
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Sevice
0,25446,4,5,2017-08-06 16:11:00,45
1,22948,48,5,2017-08-06 16:12:00,485
2,10618,0,8,2017-08-06 16:13:00,08
3,7256,9,4,2017-08-06 16:14:00,94
4,25446,48,5,2017-08-06 16:16:00,485


Adım 2: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır. Association Rule Learning uygulayabilmek için bir sepet tanımı oluşturulması gerekmektedir. Burada sepet tanımı her bir müşterinin aylık aldığı hizmetlerdir. 

Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti; 2017’in 10.ayında aldığı 9_4, 38_4 hizmetleri başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir. Bunun için öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturacağız. UserID ve yeni oluşturduğunuz date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayacağız.

In [25]:
# Öncelikle dataframemizin infosuna bakıp dtypelara göz atalım. 
df.info()
# Görüldüğü üzere createdate değişkeni object, bunun date formatına çevrilmesi gerekmektedir. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
 4   Sevice      162523 non-null  object
dtypes: int64(3), object(2)
memory usage: 6.2+ MB


In [26]:
# Create değişkenini object veri tipinden to_datetime fonksiyonu ile date'e çeviriyoruz. 
df["CreateDate"] = pd.to_datetime(df["CreateDate"])
# NEW_DATE adında bir değişken oluşturmamız gerekmekte. CreateDate'in sadece yıl ve ay verilerini alıyoruz. 
df["NEW_DATE"] = df["CreateDate"].dt.strftime("%Y-%m")
# BasketID için de bir list comprehension yapısı kullanacağız. Değerlerde dön, row 0 userıd row5 new datei ifade etmektedir. 
df["BasketID"] = [str(row[0]) + "_" + str(row[5]) for row in df.values]
df.head()


,UserId,ServiceId,CategoryId,CreateDate,Sevice,NEW_DATE,BasketID
0,25446,4,5,2017-08-06 16:11:00,45,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,485,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,08,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,94,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,485,2017-08,25446_2017-08


In [27]:
# 7256 userıd ye sahip müşterinin sepete eklediği ürünleri ve zamanlarını gözlemleyebiliyoruz. 
df[df["UserId"] == 7256 ]

,UserId,ServiceId,CategoryId,CreateDate,Sevice,NEW_DATE,BasketID
3,7256,9,4,2017-08-06 16:14:00,94,2017-08,7256_2017-08
1268,7256,46,4,2017-08-09 16:15:00,464,2017-08,7256_2017-08
9540,7256,46,4,2017-08-29 03:53:00,464,2017-08,7256_2017-08
24679,7256,9,4,2017-10-01 04:59:00,94,2017-10,7256_2017-10
24680,7256,38,4,2017-10-01 05:01:00,384,2017-10,7256_2017-10
28698,7256,9,4,2017-10-11 08:06:00,94,2017-10,7256_2017-10
65325,7256,15,1,2017-12-31 04:17:00,151,2017-12,7256_2017-12
67093,7256,2,0,2018-01-03 22:06:00,20,2018-01,7256_2018-01
70623,7256,38,4,2018-01-11 13:07:00,384,2018-01,7256_2018-01
160299,7256,18,4,2018-07-25 00:51:00,184,2018-07,7256_2018-07


### Görev 2: Birliktelik Kuralları Üretme

Adım 1: Basket ve Service pivot table oluşturma. Service stünlarda, Basket satırlarda olmalı ve içerisindeki veriler binary bir şekilde encode edilmesi gerekir. 

In [30]:
invoice_product_df = df.groupby(['BasketID', 'Sevice'])['Sevice'].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
invoice_product_df.head()
# .unstack() metodu ile basketıd satır service stüna gelmiş oluyor. 
# boşlıkları 0 ile doldurmasını fillna(0) şeklinde sağlıyoruz. 
# applymap metodu ile x 0dan büyükse 1 değilse 0 yazıyoruz. 

Sevice,08,109,1111,127,1311,14,147,151,168,175,...,456,464,477,485,491,511,67,73,85,94
BasketID,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


Adım 2: Birliktelik kurallarını oluşturuyoruz.

In [31]:
# Öncelikle burada bütün olasılıkları değerlendiriyoruz ve apriori algoritmasını çağırıyoruz.
# apriorinin içerisine biraz önce oluşturduğumuz invoice_product_df değerini gönderiyoruz. 
frequent_itemsets = apriori(invoice_product_df, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(20),(1311),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325
1,(1311),(20),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216
2,(20),(151),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833
3,(151),(20),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066
4,(151),(334),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262


In [32]:
# antecedents 1. ürünü temzil etmektedir. 
# consequents 2.ürünü temsil etmektedir. 
# antecedent support 1. ürünün tek başına gözlenme değerini,
# consequent support 2. ürünün tek başına gözlenme değerini,
# support iki ürünün birlikte satın alınma değerini,
# confidence x satın alındığında y nin satın alınma oranını göstermektedir. 
# lift x satın alındığında ynin satın alınma olasılığı lift kadar artar. 


Adım 3: arl_recommender fonksiyonunu kullanarak en son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulunuyoruz.

In [36]:
def arl_recommender(rules_df, product_id, rec_count=1): #rec_count kaç ürün tavsiye edeceğimizi gösteriyor.
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    # kuralları lifte göre büyükten kücüğe sıralar. (en uyumlu ilk ürünü yakalayabilmek için)
    # confidence'e göre de sıralanabilir insiyatife baglıdır.
    recommendation_list = [] # tavsiye edilecek ürünler için bos bir liste olusturuyoruz.
    # antecedents: X
    #items denildigi için frozenset olarak getirir. index ve hizmeti birleştirir.
    # i: index
    # product: X yani öneri isteyen hizmet
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product): # hizmetlerde(product) gez:
            if j == product_id:# eger tavsiye istenen ürün yakalanırsa:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
                # index bilgisini i ile tutuyordun bu index bilgisindeki consequents(Y) değerini recommendation_list'e ekle.

    # tavsiye listesinde tekrarlamayı önlemek için:
    # mesela 2'li 3'lü kombinasyonlarda aynı ürün tekrar düşmüş olabilir listeye gibi;
    # sözlük yapısının unique özelliginden yararlanıyoruz.
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count] # :rec_count istenen sayıya kadar tavsiye ürün getir.
arl_recommender(rules,"2_0", 1)

[]